In [7]:
import pandas as pd
import difflib
import ast
from collections import defaultdict
import re

In [8]:
df = pd.read_excel("table_1923.xlsx", names=["index", "lastname", "firstname", "org", "job", "loc", "line"])
df = df.iloc[:, 1:]

In [9]:
STREET_CAT = ["Acacias", "Bains", "Roseneck", "Beau-Rivage", "Beauregard", "Maria-Belgia", "Belle-source", "François-Bocion", "Jean-Louis-de-Bons", "Brillancourt", "Bruyères", "Cart", "Closelet", "Cour", "Crêt", "Dapples", "Église anglaise", "Pré-fleuri", "Joliette", "Délices", "Épinettes", "Fleurettes", "Mont-Tendre", "Floréal", "Fontenailles", "Fontenay", "Fraisse", "Jordils", "Funiculaire", "Grammont", "Grancy", "Grande-rive", "Grasset", "Harpe", "Navigation", "Liseron", "Milan", "Mollendruz", "Mon-loisir", "Mont-d’or", "Montriond", "Mouettes", "Ouchy", "Auguste-Pidou", "Plaines", "Port", "Pré-Fleuri", "Rod", "Rond-Point", "Tilleuls", "Servan", "Suchet", "Voltaire", "Warney"
]

In [10]:
def clean_street(word):
    ABREV = ['av.', 'r.', 'ch.', ' du ', ' des ', 'de la', 'avenue', 'rue', 'passage', 'aven', 'bd.', 'bdde', 'av ']

    if len(word) < 3:
        return ""
    
    word = word.replace(".", "")
    word = word.replace(",", "")

    word = ''.join([i for i in word if not i.isdigit()])

    for ab in ABREV:
        word = word.replace(ab, "")
    
    if len(word) > 0 and word[0] == ' ':
        word = word[1:]
    return word

In [11]:
final_streets_dict = defaultdict(list)
count = 0

for index, row in df.iterrows():
    streets_in_row = row['loc']
    row_street_list = ast.literal_eval(streets_in_row)
    for row_street in row_street_list:
        new_street = clean_street(row_street)

        result = difflib.get_close_matches(new_street, STREET_CAT, n=1, cutoff=0.65)
        if result:
            count += 1
            print(f"looking for {new_street} got {result[0]}")

            final_streets_dict[result[0]].append(index)

final_streets_dict
print(count)

looking for Montbenon got Montriond
looking for Fraisse  got Fraisse
looking for Grançy  got Grancy
looking for F„ Harpe  got Harpe
looking for Bdde Grancy  got Grancy
looking for Secretan  got Servan
looking for Closelet  got Closelet
looking for Secretan  got Servan
looking for Harpe  got Harpe
looking for Fleurettes  got Fleurettes
looking for Pont  got Port
looking for Bergières  got Bruyères
looking for de Cour  got Cour
looking for Grancy  got Grancy
looking for Grancy  got Grancy
looking for Grancy  got Grancy
looking for Grancy  got Grancy
looking for Grancy  got Grancy
looking for Fleurettes  got Fleurettes
looking for Eglise Anglaise  got Église anglaise
looking for d’Ouchy  got Ouchy
looking for Rond Point  got Rond-Point
looking for dela Harpe got Harpe
looking for Pont  got Port
looking for Dapples  got Dapples
looking for Servan  got Servan
looking for de Cour  got Cour
looking for Ed-Point  got Rond-Point
looking for Rond-Point  got Rond-Point
looking for avKond-Point  g

In [12]:
rows_list = []

for street_key in final_streets_dict.keys():

    for index in final_streets_dict[street_key]:
        row = df.iloc[index]
        new_row = {
            'cat_loc': street_key,
            'lastname': row['lastname'],
            'firstname': row['firstname'],
            'org': row['org'],
            'job': row['job'],
            'loc': row['loc'],
            'line': row['line'],
        }
        rows_list.append(new_row)

df_index_street = pd.DataFrame(rows_list)

df_index_street

,cat_loc,lastname,firstname,org,job,loc,line
0,Montriond,NaN,NaN,Abbaye de l’Arc (cercle,[],['Montbenon1.'],"Abbaye de l’Arc (cercle), Montbenon1. ET. 85G5."
1,Montriond,Cavin-Byrde,Alex,NaN,['chef d’institution'],['Montriond-le-Crêt'],"Cavin-Byrde Alex., chef d’institution,Montrion..."
2,Montriond,Chevallcy-Blane,Ernest,NaN,['ag. de policePrès'],"['Montriond, Cour']","Chevallcy-Blane Ernest, ag. de policePrès Mont..."
3,Montriond,Delacombaz,Marcel,NaN,[],['av. Mentlion 8'],"— Marcel, av. Mentlion 8."
4,Montriond,Delapierrc-Dupuis,Eugène,NaN,['électric'],"[',eli. du Crût 1, Montriond']","Delapierrc-Dupuis Eugène, électric.,eli. du Cr..."
...,...,...,...,...,...,...,...
3231,Pré-Fleuri,Tzaut,Alfred,NaN,['man'],"['Pré Fleuri, av. deCour.']","Tzaut Alfred, man., Pré Fleuri, av. deCour."
3232,Pré-Fleuri,Verrey-Wcstphal,Am,NaN,['oculiste'],"['ch.Pré Fleuri 8', 'Cab. r. Richard 3']","Verrey-Wcstphal Am., oculiste, ch.Pré Fleuri 8..."
3233,Pré-Fleuri,Viredaz-Brahier,Charles,NaN,['papeterieen gros'],"['ch. Pré Fleuri 4', 'B. av.de la Gare 23']","Viredaz-Brahier Charles, papeterieen gros, ch...."
3234,Pré-Fleuri,Wcnger,Jeanne,NaN,[],"['Pré Fleuri, Vennes']","— Jeanne, Pré Fleuri, Vennes."


In [13]:
df_index_street[df_index_street['cat_loc'] == 'Rond-Point']

,cat_loc,lastname,firstname,org,job,loc,line
1933,Rond-Point,Agner,Alfred,NaN,['opérât'],"['Rond Point 1', '.']","Agner Alfred, opérât., Rond Point 1."
1934,Rond-Point,Amarant,Paimira,NaN,['mén'],['Ed-Point 3'],"Amarant Paimira, mén., Ed-Point 3."
1935,Rond-Point,Amarant-Joly,Mar,NaN,['joum'],['av. Rond-Point 3'],"Amarant-Joly Mar., joum., av. Rond-Point 3."
1936,Rond-Point,Amblas-Trachsel,Louis,NaN,['peintre'],['av.Kond-Point 11'],"Amblas-Trachsel Louis, peintre, av.Kond-Point 11."
1937,Rond-Point,Amy-Pauli,Alexandre,NaN,['coiffeur'],['av.Rond-Point 18'],"Amy-Pauli Alexandre, coiffeur, av.Rond-Point 18."
...,...,...,...,...,...,...,...
2154,Rond-Point,Wlnkelmann-Schmld,Léon,NaN,['électric'],"[',Acacias 3, av. Rond-Point 7']","Wlnkelmann-Schmld Léon, électric.,Acacias 3, a..."
2155,Rond-Point,Wittweiler,Bertha,NaN,['ling'],['Rd-Point 7'],"Wittweiler Bertha, ling., Rd-Point 7."
2156,Rond-Point,Wood,Diane,NaN,['dame'],['Gd-Pont'],"Wood Diane, dame, Gd-Pont. G. T5188"
2157,Rond-Point,Yersin,Fanny,NaN,['pension'],['Gd-Pont 16'],"— Fanny, pension, Gd-Pont 16."


In [14]:
df_index_street.to_csv("1923_index_street.csv")

In [15]:
df_index_street_clean = pd.read_csv("1923_index_street_cleaned.csv")

# Street cleaning (we only keep relevant streets)

In [16]:
for index, row in df_index_street_clean.iterrows():
    streets_in_row = row['loc']
    row_street_list = ast.literal_eval(streets_in_row)

    filtered_street = ""

    for row_street in row_street_list:
        new_street = clean_street(row_street)
        result = difflib.get_close_matches(new_street, STREET_CAT, n=1, cutoff=0.65)

        if result:
            df_index_street_clean.loc[index, 'loc'] = row_street

In [17]:
df_index_street_clean.to_csv("1923_index_street_final.csv")

# Street number extraction

In [18]:
df_index_street_final = pd.read_csv("1923_index_street_final.csv")

In [19]:
df_index_street_final

,Unnamed: 0.1,Unnamed: 0,cat_loc,lastname,firstname,org,job,loc,line
0,0,0,Montriond,NaN,NaN,Abbaye de l’Arc (cercle,[],Montbenon1.,"Abbaye de l’Arc (cercle), Montbenon1. ET. 85G5."
1,1,1,Montriond,Cavin-Byrde,Alex,NaN,['chef d’institution'],Montriond-le-Crêt,"Cavin-Byrde Alex., chef d’institution,Montrion..."
2,2,2,Montriond,Chevallcy-Blane,Ernest,NaN,['ag. de policePrès'],"Montriond, Cour","Chevallcy-Blane Ernest, ag. de policePrès Mont..."
3,3,3,Montriond,Delacombaz,Marcel,NaN,[],av. Mentlion 8,"— Marcel, av. Mentlion 8."
4,4,4,Montriond,Delapierrc-Dupuis,Eugène,NaN,['électric'],",eli. du Crût 1, Montriond","Delapierrc-Dupuis Eugène, électric.,eli. du Cr..."
...,...,...,...,...,...,...,...,...,...
3231,3231,3231,Pré-Fleuri,Tzaut,Alfred,NaN,['man'],"Pré Fleuri, av. deCour.","Tzaut Alfred, man., Pré Fleuri, av. deCour."
3232,3232,3232,Pré-Fleuri,Verrey-Wcstphal,Am,NaN,['oculiste'],ch.Pré Fleuri 8,"Verrey-Wcstphal Am., oculiste, ch.Pré Fleuri 8..."
3233,3233,3233,Pré-Fleuri,Viredaz-Brahier,Charles,NaN,['papeterieen gros'],ch. Pré Fleuri 4,"Viredaz-Brahier Charles, papeterieen gros, ch...."
3234,3234,3234,Pré-Fleuri,Wcnger,Jeanne,NaN,[],"Pré Fleuri, Vennes","— Jeanne, Pré Fleuri, Vennes."


In [20]:
df_index_street_final.drop(columns=[df_index_street_final.columns[0]], inplace=True)
df_index_street_final.drop(columns=[df_index_street_final.columns[0]], inplace=True)
df_index_street_final

,cat_loc,lastname,firstname,org,job,loc,line
0,Montriond,NaN,NaN,Abbaye de l’Arc (cercle,[],Montbenon1.,"Abbaye de l’Arc (cercle), Montbenon1. ET. 85G5."
1,Montriond,Cavin-Byrde,Alex,NaN,['chef d’institution'],Montriond-le-Crêt,"Cavin-Byrde Alex., chef d’institution,Montrion..."
2,Montriond,Chevallcy-Blane,Ernest,NaN,['ag. de policePrès'],"Montriond, Cour","Chevallcy-Blane Ernest, ag. de policePrès Mont..."
3,Montriond,Delacombaz,Marcel,NaN,[],av. Mentlion 8,"— Marcel, av. Mentlion 8."
4,Montriond,Delapierrc-Dupuis,Eugène,NaN,['électric'],",eli. du Crût 1, Montriond","Delapierrc-Dupuis Eugène, électric.,eli. du Cr..."
...,...,...,...,...,...,...,...
3231,Pré-Fleuri,Tzaut,Alfred,NaN,['man'],"Pré Fleuri, av. deCour.","Tzaut Alfred, man., Pré Fleuri, av. deCour."
3232,Pré-Fleuri,Verrey-Wcstphal,Am,NaN,['oculiste'],ch.Pré Fleuri 8,"Verrey-Wcstphal Am., oculiste, ch.Pré Fleuri 8..."
3233,Pré-Fleuri,Viredaz-Brahier,Charles,NaN,['papeterieen gros'],ch. Pré Fleuri 4,"Viredaz-Brahier Charles, papeterieen gros, ch...."
3234,Pré-Fleuri,Wcnger,Jeanne,NaN,[],"Pré Fleuri, Vennes","— Jeanne, Pré Fleuri, Vennes."


In [21]:
street_number = []

max_iter = 0

for index, row in df_index_street_final.iterrows():
    street_in_row = row['loc']

    digits_group = re.findall('\d+', street_in_row)

    # digits_group should only have one number (ie. a list of one element) or nothing if there is no digits

    if len(digits_group) == 0:
        street_number.append(0) # street numbers cannot be 0 and from what I checked, I did not detect OCR failure
    elif len(digits_group) > 1:
        print("should not happen", digits_group)
        raise Exception
    else:
        street_number.append(int(digits_group[0]))

In [22]:
df_index_street_final.insert(len(df_index_street_final.columns) - 1, "street_num", street_number)

In [23]:
df_index_street_final.to_csv('1923_index_street_final_best.csv')

In [24]:
df_index_street_final

,cat_loc,lastname,firstname,org,job,loc,street_num,line
0,Montriond,NaN,NaN,Abbaye de l’Arc (cercle,[],Montbenon1.,1,"Abbaye de l’Arc (cercle), Montbenon1. ET. 85G5."
1,Montriond,Cavin-Byrde,Alex,NaN,['chef d’institution'],Montriond-le-Crêt,0,"Cavin-Byrde Alex., chef d’institution,Montrion..."
2,Montriond,Chevallcy-Blane,Ernest,NaN,['ag. de policePrès'],"Montriond, Cour",0,"Chevallcy-Blane Ernest, ag. de policePrès Mont..."
3,Montriond,Delacombaz,Marcel,NaN,[],av. Mentlion 8,8,"— Marcel, av. Mentlion 8."
4,Montriond,Delapierrc-Dupuis,Eugène,NaN,['électric'],",eli. du Crût 1, Montriond",1,"Delapierrc-Dupuis Eugène, électric.,eli. du Cr..."
...,...,...,...,...,...,...,...,...
3231,Pré-Fleuri,Tzaut,Alfred,NaN,['man'],"Pré Fleuri, av. deCour.",0,"Tzaut Alfred, man., Pré Fleuri, av. deCour."
3232,Pré-Fleuri,Verrey-Wcstphal,Am,NaN,['oculiste'],ch.Pré Fleuri 8,8,"Verrey-Wcstphal Am., oculiste, ch.Pré Fleuri 8..."
3233,Pré-Fleuri,Viredaz-Brahier,Charles,NaN,['papeterieen gros'],ch. Pré Fleuri 4,4,"Viredaz-Brahier Charles, papeterieen gros, ch...."
3234,Pré-Fleuri,Wcnger,Jeanne,NaN,[],"Pré Fleuri, Vennes",0,"— Jeanne, Pré Fleuri, Vennes."


# Occupation extraction

In [25]:
df_index_street_final_best = pd.read_csv("1923_index_street_final_best.csv", index_col=0)

In [26]:
df_index_street_final_best

,cat_loc,lastname,firstname,org,job,loc,street_num,line
0,Montriond,NaN,NaN,Abbaye de l’Arc (cercle,[],Montbenon1.,1,"Abbaye de l’Arc (cercle), Montbenon1. ET. 85G5."
1,Montriond,Cavin-Byrde,Alex,NaN,['chef d’institution'],Montriond-le-Crêt,0,"Cavin-Byrde Alex., chef d’institution,Montrion..."
2,Montriond,Chevallcy-Blane,Ernest,NaN,['ag. de policePrès'],"Montriond, Cour",0,"Chevallcy-Blane Ernest, ag. de policePrès Mont..."
3,Montriond,Delacombaz,Marcel,NaN,[],av. Mentlion 8,8,"— Marcel, av. Mentlion 8."
4,Montriond,Delapierrc-Dupuis,Eugène,NaN,['électric'],",eli. du Crût 1, Montriond",1,"Delapierrc-Dupuis Eugène, électric.,eli. du Cr..."
...,...,...,...,...,...,...,...,...
3231,Pré-Fleuri,Tzaut,Alfred,NaN,['man'],"Pré Fleuri, av. deCour.",0,"Tzaut Alfred, man., Pré Fleuri, av. deCour."
3232,Pré-Fleuri,Verrey-Wcstphal,Am,NaN,['oculiste'],ch.Pré Fleuri 8,8,"Verrey-Wcstphal Am., oculiste, ch.Pré Fleuri 8..."
3233,Pré-Fleuri,Viredaz-Brahier,Charles,NaN,['papeterieen gros'],ch. Pré Fleuri 4,4,"Viredaz-Brahier Charles, papeterieen gros, ch...."
3234,Pré-Fleuri,Wcnger,Jeanne,NaN,[],"Pré Fleuri, Vennes",0,"— Jeanne, Pré Fleuri, Vennes."


In [27]:
JOB_CAT_ABRV = {
                'chef': ['chef'],
                'policier': ['ag. de olice', 'police', 'polic'],
                'électricien': ['électric', 'élec'],
                'architecte': ['arch'],
                'professeur': ['prof', "prof d'école"],
                'couturier': ['cout'],
                'tapissier': ['tapiss', 'tap'],
                'agriculteur': ['agri', 'agricult'],
                'directeur': ['d.', 'dir'],
                'ménagère': ['ménag', 'men', 'mén'],
                'serrurier': ['serrur', 'serrar'],
                'commerçant': ['comm', 'commerc', 'commercant'],
                'maçon': ['macon'],
                'évangéliste': ['évang', 'évan'],
                'comptable': ['compt'],
                'veuve': ['vve', 'veuv'],
                'négociant': ['nég', 'négo'],
                'conducteur CFF': ['cond CFF', 'condu CFF', 'conduc CFF', 'conducteur CFF'],
                'étalagiste': ['étalag', 'étalag', 'étal'],
                'confiseur': ['confis', 'confisr'],
                'employé': ['emp', 'empi', 'emplo', 'empl', 'employée'],
                'portier CFF': ['port CFF', 'portier CFF'],
                'portier': ['porte'],
                'voyagiste': ['voyagiste'],
                'gendarme': ['de gend', 'gendarm', 'gendarmerie', 'gend', 'sergent de gendarmerie', 'cap. de gend', 'capitaine gendarmerie'],
                'chauffeur': ['chauff', 'chauff'],
                'chauffeur CFF': ['chauff CFF', 'chauff CFF'],
                'céramiste': ['céram', 'cérami'],
                'brigadier de sûreté': ['brig', 'briga', 'brig de sûreté'],
                'assureur': ['assur', 'assuran', 'assureu'],
                'mécanicien': ['mécan', 'méca', 'mécano'],
                'mécanicien CFF': ['mécan CFF', 'méca CFF', 'mécano CFF'],
                'manœuvre': ['man', 'man', 'manœuv', 'manoeuvre'],
                'modiste': ['modis', 'mod'],
                'télégraphiste': ['télég', 'télégraphi', 'télégraphe'],
                'instituteur': ['instit', 'insituteu', 'insti', 'institutrice'],
                'représentant': ['repré', 'repr', 'représent'],
                'employé tramway': ['employédetrams', 'empl de tram', 'empl de trams'],
                "consul d'Italie": ["consul ditalie"],
                "vice-consul d'Italie": ["vice-consul ditalie"],
                'banquier': ['banq'],
                'dentiste': ['denti', 'dent'],
                'tabagiste': ['tabac', 'tabacs'],
                'étudiant': ['étud', 'étudia'],
                'tonnelier': ['tonnel'],
                'encaisseur': ['encaiss'],
                'cordonnier': ['cordonn', 'cordon', 'cordo'],
                'gérant': ['géran'],
                'ouvrier': ['ouvr'],
                'coiffeur': ['coiff', 'coiffeuse'],
                'tailleur': ['taill', 'tailleuse'],
                "inspecteur d'assurances": ["inspecteur d'assur"],
                'inspecteur CFF': ['inspect.CFF', 'inspect. CFF'],
                'rentier': ['rentière', 'renti', 'rent'],
                'tanneur': ['tann'],
                'vigneron': ['vign', 'vinger'],
                'commis': ['commi'],
                'commis CFF': ['com CFF', 'commis C.F.F.', 'com C.F.F.'],
                'pharmacien': ['pharmaa'],
                'couvreur': ['couvr'],
                'cuisinier': ['cuisin'],
                'restaurateur': ['restaur', 'g de rest'],
                'peintre': ['peint'],
                'horloger': ['horl', 'horloge'],
                'laitier': ['g laitier', 'laiterie'],
                'sommelier': ['sommel'],
                'violoncelliste': ['violoncel'],
                'sellier': ['sell'],
                'boulanger': ['boulan', 'boul'],
                'corsetière': ['corsetier'],
                'dactylographe': ['dactyl', 'dactylo'],
                'secrétaire': ['secrét'],
                'secrétaire CFF': ['secrét CFF'],
                'employé CFF': ['empi CFF', 'employ CFF'],
                'employé de commerce': ['emplo de commerce', 'empi de commerce'],
                'héliographe': ['hélio'],
                'médecin': ['méd', 'médec'],
                'géomètre': ['géo', 'géom'],
                'typographe': ['typo'],
                'menuisier': ['menu'],
                'consul de Belgique': ['consu de belgique'],
                'ingénieur': ['ingé', 'ing'],
                'domestique': ['domest'],
                'mouleur': ['moul'],
                'nettoyeur': ['netto', 'nett'],
                'artiste': ['art'],
                'publiciste': ['publi', 'publicité'],
                'concierge': ['concier', 'concierg'],
                'ramoneur': ['ramone'],
                'maréchal-ferrant': ['maréchal'],
                'pianiste': ['pian'],
                'chimiste': ['chim', 'chimi'],
                'livreur': ['livr', 'livraison'],
                'avocat': ['avoc'],
                'charcutier': ['charcut'],
                'épicier': ['épic'],
                'cafetier': ['cafet'],
                'lingère': ['ling', 'linger', 'linge'],
                'blanchisseuse': ['blanchis', 'blanchisseur', 'blanchiss'],
                'ébéniste': ['ében'],
                'fourreur': ['four', 'fourrures'],
                'relieur': ['reli', 'relieuse'],
                'teinturier': ['teint', 'tentur', 'teintures', 'teintureries'],
                'drogiste': ['drog', 'droguerie'],
                'commis postal': ['com. postal', 'comm. postal', 'commispostal'],
                'caissière': ['caisse', 'caissier'],
                'compositeur': ['compo', 'composit'],
                'charron': ['charr'],
                "agent d’affaires": ["agent d'aff"],
                'jounaliste': ['journal', 'journ'],
                'brodeuse': ['brod', 'brodeur'],
                'vendeur': ['vend', 'vendeuse'],
                'emballeur': ['emball', 'emballages'],
                'éditeur': ['éditeuse', 'édit', 'édition'],
                'jardinier': ['jardin', 'jardi'],
                'employé du gaz': ['empi du gaz', 'gaz'],
                'sténographe': ['sténo', 'steno'],
                "conseiller d'état": ["conseiller d'etat"],
                'attaché': ['attach', 'attachée'],
                'courtier': ['court', 'courtage'],
                'coutelier': ['couteaux', 'coutellerie'],
                'cocher': ['coch'],
                'employé des douanes': ['douanes', 'empi douanes', 'garde-front'],
                'administrateur': ['admin'],
                'fleuriste': ['fleurs', 'fleur'],
                'industriel': ['indust', 'indus'],
                'curé': ['cure', 'cur'],
                'imprimeur': ['impri', 'imprimerie'],
                'orfèvre': ['orfè', 'orf'],
                'oculiste': ['ocul'],
                'bijoutier': ['bijou', 'bijoux', 'bijoutière'],
                "chef d’institution": ["chef d'institut"],
                'papetier': ['papier', 'papeterie'],
                'religieux': ['religieuse'],
                'fonctionnaire': ['fonct', 'fonct postal', 'fonctionnaire postal'],
                'primeurs': ['prim', 'primeur'],
                'combustibles': ['combust'],
                'femme de chambre': ['f. de ch.', 'f de ch', 'f. de ch', 'f. de chambre'],
                'bottier': ['bottes', 'botte', 'bottière'],
                'sous-chef de gare': ['ss-chef de gare', 's.-chef de gare'],
                'employé postal': ['empi postal', 'employ postal', 'empl postal', 'employé de poste', 'employé des postes', 'emp. post'],
                'employé de banque': ['empi,de banque', 'emp. de banque'],
                'd. de mag': ['d de mag'],
                'professeur de piano et de chant': ['prof, depiano et chant'],
                'professeur de piano': ['prof, depiano', 'prof. de piano'],
                'professeur de musique': ['prof, musique'],
                'directeur du Crédit Suisse': ['dir Crédit Suisse'],
                'négociant en vins': ['négo. en vins', 'négoc. en vins'],
                'chef bureau CFF': ['chef bur. CFF'],
                'chef de train CFF': ['chef detrain CFF'],
                'masseur': ['masseuse', 'massouse'],
                'pension': ['pensio'],
                'ingénieur-professeur': ['ing. prof', 'ingé. prof'],
                'gouvernante': ['gouv'],
                'boucher': ['bouch'],
                'manœuvre CFF': ['man CFF'],
                'substitut du procureur génénéral': ['substitut duProcur. gén'],
                'facteur': ['fact'],
                'gypsier': ['gyps', 'gypse'],
                'faïences': ['faiences', 'faïence'],
                'commissionnaire': ['commissionnair'],
                'technicien': ['techni', 'techn'],
                'secrétaire à la direction de police': ['secrét. à ladir. de police'],
                'garde-malade': ['g.-mal.', 'g.-malade'],
                'employé de pharmacie': ['empi de phrmacie', 'employée de pharmacie', 'emp. de phar', 'emp. de pharmacie'],
                'tréfileur': ['tréfil', 'tréfileuse', 'tréfilage'],
                'professeur de peinture': ['prof. de peinture'],
                'directeur de pensionnat': ['dir. de pensionnat'],
                'droguiste': ['drog.', 'drogue', 'droguerie', 'droguerie en gros'],
                'vitrier': ['vitre', 'vit.', 'vitrerie'],
                'vulcanisateur': ['vulcan', 'vulcanisation', 'vulcanisatrice'],
                'chef de bureau postal': ['chef de bureaupostal'],
                'sculpteur': ['sculpt', 'sculpteuse', 'sculpterie'],
                'bouclier': ['boucle', 'boucl.'],
                'appareilleur': ['appreill'],
                'voyageur': ['voyageuse', 'voyag.'],
                'contrôleur': ['controleur', 'contrôl', 'contrôle', 'controle'],
                'plieur': ['plieuse', 'pli', 'pliage'],
                'charretier': ['charrette', 'charretière'],
                'inspecteur des bâtiments': ['inspect. des bâtiments'],
                'musicien': ['musique', 'musicien'],
                'batelier': ['batelière'],
                'ouvrier CFF': ['ouvrière CFF', 'ouvr. CFF'],
                'postillon': ['postill.'],
                'professeur de dessin': ['prof. dessin'],
                'briquetier': ['briques'],
                'pierriste': ['pierres', 'pierre'],
                'magasinier': ['magas.', 'magasin'],
                'charpentier': ['charpente'],
                'commissionnaire': ['commission'],
                'dessinateur': ['dessin'],
                'brasseur': ['brasseuse'],
                'électricien CFF': ['électr. CFF'],
                'constructeur de bateaux': ['const, de bateaux'],
                'machiniste': ['machine'],
                'lithographe': ['lithogr.', 'lith'],
                'monteur': ['monteuse', 'montage'],
                'opticien': ['optique', 'opticienne'],
                "huissier du Conseil d’Etat": ["huissier duConseil d’Etat"],
                'désinfecteur': ['désinfection', 'désinfect'],
                'pâtissier': ['pâtiss'],
                'infirmier': ['infirmière', 'infirm'],
                'pasteur': ['past'],
                'liftier': ['lift']
}

for cat_abrv in JOB_CAT_ABRV.keys():
    new_list = JOB_CAT_ABRV[cat_abrv]
    new_list.append(cat_abrv)
    JOB_CAT_ABRV[cat_abrv] = new_list

JOB_CAT_FULL = JOB_CAT_ABRV.keys()

In [28]:
len(JOB_CAT_ABRV)

203

In [6]:
for e in JOB_CAT_ABRV.keys():
    print(e)

NameError: name 'JOB_CAT_ABRV' is not defined

In [ ]:
len(JOB_CAT_ABRV)

203

In [ ]:
def clean_job(job):
    job = job.replace(".", "")
    job = job.replace(",", "")
    job = job.replace("[", "")
    job = job.replace("]", "")
    job = job.replace('"', "")

    return job

In [ ]:
final_job_list = list()
job_cat_values = [x for xs in list(JOB_CAT_ABRV.values()) for x in xs]

for index, row in df_index_street_final_best.iterrows():
    row_job = row['job']

    cleaned_job = clean_job(row_job)
    
    if len(cleaned_job) > 0:
        result = difflib.get_close_matches(cleaned_job, job_cat_values, n=1)
        print(result)
        if result:
            print(f"have {cleaned_job}, got {result[0]}")
            for key, list_job in JOB_CAT_ABRV.items():
                if result[0] in list_job:
                    print(f"then category {key}")
                    final_job_list.append(key)
        else:
            final_job_list.append(None)
    else:
        final_job_list.append(None)


['chef d’institution']
have 'chef d’institution', got chef d’institution
then category chef d’institution
['ag. de olice']
have 'ag de policePrès', got ag. de olice
then category policier
['électric']
have 'électric', got électric
then category électricien
['arch']
have 'arch', got arch
then category architecte
['prof']
have 'prof', got prof
then category professeur
['cout']
have 'cout', got cout
then category couturier
['tapiss']
have 'tapiss', got tapiss
then category tapissier
['teinturier']
have 'tnupier', got teinturier
then category teinturier
['agri']
have 'agric', got agri
then category agriculteur
['ménag']
have 'ménag', got ménag
then category ménagère
[]
['serrurier']
have 'serrurier', got serrurier
then category serrurier
['comm']
have 'comm', got comm
then category commerçant
['agricult']
have 'agricult', got agricult
then category agriculteur
[]
['maçon']
have 'maçon', got maçon
then category maçon
['évangéliste']
have 'évangéliste', got évangéliste
then category évangéli

In [ ]:
df_index_street_final_best.insert(1, 'cat_job', final_job_list)
df_index_street_final_best

,cat_loc,cat_job,lastname,firstname,org,job,loc,street_num,line
0,Montriond,None,NaN,NaN,Abbaye de l’Arc (cercle,[],Montbenon1.,1,"Abbaye de l’Arc (cercle), Montbenon1. ET. 85G5."
1,Montriond,chef d’institution,Cavin-Byrde,Alex,NaN,['chef d’institution'],Montriond-le-Crêt,0,"Cavin-Byrde Alex., chef d’institution,Montrion..."
2,Montriond,policier,Chevallcy-Blane,Ernest,NaN,['ag. de policePrès'],"Montriond, Cour",0,"Chevallcy-Blane Ernest, ag. de policePrès Mont..."
3,Montriond,None,Delacombaz,Marcel,NaN,[],av. Mentlion 8,8,"— Marcel, av. Mentlion 8."
4,Montriond,électricien,Delapierrc-Dupuis,Eugène,NaN,['électric'],",eli. du Crût 1, Montriond",1,"Delapierrc-Dupuis Eugène, électric.,eli. du Cr..."
...,...,...,...,...,...,...,...,...,...
3231,Pré-Fleuri,manœuvre,Tzaut,Alfred,NaN,['man'],"Pré Fleuri, av. deCour.",0,"Tzaut Alfred, man., Pré Fleuri, av. deCour."
3232,Pré-Fleuri,oculiste,Verrey-Wcstphal,Am,NaN,['oculiste'],ch.Pré Fleuri 8,8,"Verrey-Wcstphal Am., oculiste, ch.Pré Fleuri 8..."
3233,Pré-Fleuri,papetier,Viredaz-Brahier,Charles,NaN,['papeterieen gros'],ch. Pré Fleuri 4,4,"Viredaz-Brahier Charles, papeterieen gros, ch...."
3234,Pré-Fleuri,None,Wcnger,Jeanne,NaN,[],"Pré Fleuri, Vennes",0,"— Jeanne, Pré Fleuri, Vennes."


In [ ]:
print(df_index_street_final_best['cat_job'].count())

# we have more than half with a job


1933


In [ ]:
df_index_street_final_best.to_csv('1923_index_street_cat_jobs.csv', index=False)

In [29]:
df_super = pd.read_csv('1923_index_street_cat_jobs.csv')

In [37]:
jobs = df_super[~df_super['cat_job'].isna()]

print(f"number of jobs identified = {len(jobs)} out of {len(df_super)}")

number of jobs identified = 1826 out of 2828
